In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

from array import array

import datetime
from datetime import timedelta, date #for time duration calculations
from dateutil.parser import parse #for fuzzy finding year

import pickle #for saving output files, pickles
from sys import stdout
import time #for time.sleep function to delay calls
from tqdm import tqdm #for updating loop
#from os import listdir
#from os.path import isfile, join
import glob #pattern matching and expansion.

In [ ]:
intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

# Function convert seconds into day.decimal  
def ConvertSectoDay(n): 
    day = n // (24 * 3600) 
    #print(day) #keep day
    n = n % (24 * 3600) 
    daydec=(n/86400) # add this to day
    addem=day+daydec
    #https://stackoverflow.com/a/48812729/1602288
    holder='{:g}'.format(float('{:.{p}g}'.format(addem, p=5)))
    return(float(holder))

In [ ]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.sql import table, column, select, update, insert
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import pandas as pd

#In Python: Define your username and password used above. I've defined the database name (we're 
#using a dataset on births, so I call it birth_db). 
dbname = 'donors_db'
username = 'xxxx'#enter username
pswd = 'xxxx'#enter password

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print(engine.url)
# Replace localhost with IP address if accessing a remote server

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
print(engine.url)


In [ ]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [ ]:
### query: from historical data
merge_query = """
SELECT * FROM merge_projects;
"""

mergedframe = pd.read_sql_query(merge_query,con)

In [ ]:
mergedframe = mergedframe[mergedframe.funding_status == 'completed']
mergedframe.head(2)

In [ ]:
def elapsedseconds(posted, completed):
    formatuse = '%Y-%m-%d %H:%M:%S' # The format: see down this page:https://docs.python.org/3/library/datetime.html
    otherformat = '%Y-%m-%d'
    
    try:
        clock = datetime.datetime.strptime(completed,formatuse) 
    except:
        clock = datetime.datetime.strptime(completed,otherformat) 
    
    try:
        startclock = datetime.datetime.strptime(posted,formatuse)
    except:
        startclock = datetime.datetime.strptime(posted,otherformat)
        
    elapsed = (clock-startclock).total_seconds()
    return(elapsed)
    

In [ ]:
#https://stackoverflow.com/a/40353780/1602288
mergedframe['latency_to_funded'] = mergedframe.apply(lambda row: elapsedseconds(row['date_posted'],row['date_completed']),axis=1)
mergedframe['days_to_funding'] = mergedframe.apply(lambda row: ConvertSectoDay(row.latency_to_funded),axis=1)
pd.set_option('display.max_columns', None)
mergedframe.head()

In [ ]:
##save new merged
#mergedframe.to_sql('merge_time', engine, if_exists='append')

In [ ]:
sns.set_context("poster", font_scale=1.3)

fig, ax = plt.subplots(figsize=(12, 8))
sns.distplot(mergedframe["days_to_funding"].dropna())
fig.tight_layout()

In [ ]:
print("Avg time to funding: "+str(display_time(mergedframe['latency_to_funded'].mean(),3)))
print("Avg time to funding: "+str(mergedframe['days_to_funding'].mean())+" days")

In [ ]:
neweval = mergedframe['days_to_funding'].describe()
readval = pd.Series([display_time(each,3) for each in neweval])
print((neweval))

print((readval))


In [ ]:
mergedframe = mergedframe[mergedframe.days_to_funding < 200]
mergedframe.shape

In [ ]:
x = mergedframe.total_price_excluding_optional_support
y = mergedframe.days_to_funding


In [ ]:
sns.set_context("poster", font_scale=1.3)

fig, ax = plt.subplots(figsize=(12, 8))


g =sns.scatterplot(x="total_price_excluding_optional_support", y="days_to_funding",
              hue="primary_focus_subject",alpha=0.1,
              data=mergedframe,legend=False);
g.set(xscale="log");

fig.tight_layout()

In [ ]:
#pd.mergedframe('display.max_columns', None)
list(mergedframe['fulfillmentTrailer'])
#mergedframe.head(2)

In [ ]:
### Close communication with the database
con.close()